<a href="https://colab.research.google.com/github/cindy12651269/mask_api/blob/cindy12651269%2Fnote/mask_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json

# 利用 requests 對 API 來源發送一個請求
url="https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json"
response=requests.get(url)

# 將請求回應的內容存成一個字串格式
response_text=response.text

# 將長得像 json 格式的字串解析成字典或列表
data=json.loads(response_text)


{'臺北市': 339, '': 49, '高雄市': 422, '臺中市': 420, '臺南市': 271, '基隆市': 57, '新竹市': 39, '嘉義市': 67, '新北市': 501, '桃園市': 264, '新竹縣': 46, '宜蘭縣': 76, '苗栗縣': 56, '彰化縣': 179, '南投縣': 67, '雲林縣': 129, '嘉義縣': 84, '屏東縣': 140, '澎湖縣': 11, '花蓮縣': 46, '臺東縣': 23, '金門縣': 6, '連江縣': 1}
{'連江縣': 7930, '澎湖縣': 12600, '金門縣': 15510, '臺東縣': 28510, '新竹市': 45630, '新竹縣': 51350, '花蓮縣': 57330, '': 69210, '基隆市': 80100, '南投縣': 91240, '宜蘭縣': 106900, '嘉義市': 107840, '嘉義縣': 113180, '苗栗縣': 131080, '屏東縣': 184250, '雲林縣': 196990, '彰化縣': 219400, '臺北市': 322490, '桃園市': 331940, '臺南市': 393900, '高雄市': 424170, '臺中市': 536710, '新北市': 670270}


In [ ]:
med_count = {}

# 填入欄位名稱
for d in data['features']:
    conunty = d['properties']['county']
    if conunty not in med_count:
         med_count[conunty] = 0
    med_count[conunty] +=1

print(med_count)
# {'台北市': 123, '新北市': 456 ...}

{'臺北市': 339, '': 49, '高雄市': 422, '臺中市': 420, '臺南市': 271, '基隆市': 57, '新竹市': 39, '嘉義市': 67, '新北市': 501, '桃園市': 264, '新竹縣': 46, '宜蘭縣': 76, '苗栗縣': 56, '彰化縣': 179, '南投縣': 67, '雲林縣': 129, '嘉義縣': 84, '屏東縣': 140, '澎湖縣': 11, '花蓮縣': 46, '臺東縣': 23, '金門縣': 6, '連江縣': 1}


In [ ]:
mask_count = {}

# 填入欄位名稱
for d in data['features']:
    county = d['properties']['county']
    adult_masks = d['properties']['mask_adult']
    if county not in mask_count:
        mask_count[county] = 0
    mask_count[county] += adult_masks

# 將結果從大到小排列
mask_count = dict(sorted(mask_count.items(), key=lambda item: item[1]))

print(mask_count)
# {'台北市': 12345, '新北市': 45678 ...}

{'連江縣': 7930, '澎湖縣': 12600, '金門縣': 15510, '臺東縣': 28510, '新竹市': 45630, '新竹縣': 51350, '花蓮縣': 57330, '': 69210, '基隆市': 80100, '南投縣': 91240, '宜蘭縣': 106900, '嘉義市': 107840, '嘉義縣': 113180, '苗栗縣': 131080, '屏東縣': 184250, '雲林縣': 196990, '彰化縣': 219400, '臺北市': 322490, '桃園市': 331940, '臺南市': 393900, '高雄市': 424170, '臺中市': 536710, '新北市': 670270}


In [11]:
#將昨天計算的「各地區的藥局數量」存入資料庫的 `pharmacies` 表格，欄位包含「地區」、「數量」、「新增時間」
import requests
import json
import sqlite3
import datetime


conn = sqlite3.connect('example.db')
c = conn.cursor()

# 新增且清空資料表
c.execute('''CREATE TABLE IF NOT EXISTS pharmacies
             (city text, counts text, createdAt datetime)''')
c.execute('''DELETE FROM pharmacies''')
conn.commit()

# 新增資料
url="https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json"
response=requests.get(url)

response_text=response.text

data=json.loads(response_text)

#將計算的「各地區的藥局數量」存入資料庫的 `pharmacies` 表格，欄位包含「地區」、「數量」、「新增時間」
# 蒐集資料

med_count = {}
for d in data['features']:
    conunty = d['properties']['county']
    if conunty not in med_count:
         med_count[conunty] = 0

    else:
        med_count[conunty] += 1

# 將每一筆資料分別取出來新增到資料庫
for city, counts in med_count.items():
  t = datetime.datetime.now()
  print(f"INSERT INTO pharmacies VALUES ('{city}', {counts}, '{t}')")
  c.execute(f"INSERT INTO pharmacies VALUES ('{city}', {counts}, '{t}')")
  conn.commit()

# 查詢資料

c.execute("SELECT * FROM pharmacies")
print(c.fetchall())

conn.commit()
conn.close()

INSERT INTO pharmacies VALUES ('臺北市', 338, '2024-06-30 02:02:40.471249')
INSERT INTO pharmacies VALUES ('', 48, '2024-06-30 02:02:40.502505')
INSERT INTO pharmacies VALUES ('高雄市', 421, '2024-06-30 02:02:40.514879')
INSERT INTO pharmacies VALUES ('臺中市', 419, '2024-06-30 02:02:40.525707')
INSERT INTO pharmacies VALUES ('臺南市', 270, '2024-06-30 02:02:40.536411')
INSERT INTO pharmacies VALUES ('基隆市', 56, '2024-06-30 02:02:40.546130')
INSERT INTO pharmacies VALUES ('新竹市', 38, '2024-06-30 02:02:40.556185')
INSERT INTO pharmacies VALUES ('嘉義市', 66, '2024-06-30 02:02:40.566515')
INSERT INTO pharmacies VALUES ('新北市', 500, '2024-06-30 02:02:40.576261')
INSERT INTO pharmacies VALUES ('桃園市', 263, '2024-06-30 02:02:40.586124')
INSERT INTO pharmacies VALUES ('新竹縣', 45, '2024-06-30 02:02:40.595952')
INSERT INTO pharmacies VALUES ('宜蘭縣', 75, '2024-06-30 02:02:40.604957')
INSERT INTO pharmacies VALUES ('苗栗縣', 55, '2024-06-30 02:02:40.614058')
INSERT INTO pharmacies VALUES ('彰化縣', 178, '2024-06-30 02:02:

In [10]:
#接下來請將昨天計算的「各地區的剩餘口罩」存入資料庫的 `masks` 表格，欄位包含「地區」、「剩餘口罩數量」、「新增時間」
import sqlite3
import datetime

conn = sqlite3.connect('example.db')
c = conn.cursor()

# 新增且清空資料表
c.execute('''CREATE TABLE IF NOT EXISTS masks(city text, counts text, createdAt datetime)''') #建立Table masks，並設定欄位，以及欄位的資料型態。
c.execute('''DELETE FROM masks''') #清空Table
conn.commit()


# 新增資料
mask_count = {} # 把收集資料部分放在這邊

for d in data['features']:
    county = d['properties']['county']
    mask_adult = d['properties']['mask_adult']
    if county not in mask_count:
        mask_count[county] = 0

    else:
        mask_count[county] += mask_adult #計算成人口罩數量
# mask_count = dict(sorted(mask_count.items(), key=lambda item: item[1], reverse=True)) # 將結果從大到小排列


# 將每一筆資料分別取出來新增到資料庫
for city, counts in mask_count.items():
  t = datetime.datetime.now()
#   print(f"INSERT INTO masks VALUES ('{city}', {counts}, '{t}')")
  c.execute(f"INSERT INTO masks VALUES ('{city}', {counts}, '{t}')")
  conn.commit()

# 查詢資料
c.execute("SELECT * FROM masks")
print(c.fetchall())

conn.commit()
conn.close()

[('臺北市', '322490', '2024-06-30 01:49:47.700741'), ('', '68040', '2024-06-30 01:49:47.711378'), ('高雄市', '424170', '2024-06-30 01:49:47.719882'), ('臺中市', '535430', '2024-06-30 01:49:47.728092'), ('臺南市', '392430', '2024-06-30 01:49:47.735171'), ('基隆市', '79840', '2024-06-30 01:49:47.742993'), ('新竹市', '45630', '2024-06-30 01:49:47.750280'), ('嘉義市', '105870', '2024-06-30 01:49:47.757643'), ('新北市', '670150', '2024-06-30 01:49:47.764902'), ('桃園市', '330010', '2024-06-30 01:49:47.772457'), ('新竹縣', '51280', '2024-06-30 01:49:47.779597'), ('宜蘭縣', '105830', '2024-06-30 01:49:47.786966'), ('苗栗縣', '119500', '2024-06-30 01:49:47.795244'), ('彰化縣', '218090', '2024-06-30 01:49:47.802149'), ('南投縣', '81680', '2024-06-30 01:49:47.809719'), ('雲林縣', '195540', '2024-06-30 01:49:47.817108'), ('嘉義縣', '112190', '2024-06-30 01:49:47.824736'), ('屏東縣', '181480', '2024-06-30 01:49:47.832179'), ('澎湖縣', '11350', '2024-06-30 01:49:47.839320'), ('花蓮縣', '56570', '2024-06-30 01:49:47.846290'), ('臺東縣', '27720', '2024-06-30 